In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Rescaling, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import Nadam
from keras.optimizers import Ftrl
# from keras.optimizers import ProximalSGD
from keras.optimizers import Adamax

In [3]:
img_height,img_width=224,224
batch_size=32
data_dir = "/kaggle/input/apple-disease-dataset-original/Original Data/"

In [4]:
# Creating an ImageDataGenerator with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split
)

In [5]:
# Creating an ImageDataGenerator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split
)

In [6]:
train_generator = train_datagen.flow_from_directory(
    data_dir + "train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

Found 6218 images belonging to 4 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    data_dir + "train",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 1553 images belonging to 4 classes.


In [8]:
# Test dataset generator (without data augmentation)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    data_dir + "test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical'
)

Found 1943 images belonging to 4 classes.


In [9]:
# Get the class names
class_names = list(train_generator.class_indices.keys())
print("Class Names:", class_names)

Class Names: ['Apple Scab', 'Black Rot', 'Cedar Apple Rust', 'Healthy']


In [10]:
VGG16_model = Sequential()

pretrained_model = tf.keras.applications.VGG16(
                    include_top=False,
                    input_shape=(224, 224, 3),
                    pooling='avg',
                    classes=4,
                    weights='imagenet'
                   )

for layer in pretrained_model.layers:
    layer.trainable = False

VGG16_model.add(pretrained_model)
VGG16_model.add(Flatten())
VGG16_model.add(BatchNormalization())
VGG16_model.add(Dense(512, activation='relu'))
VGG16_model.add(BatchNormalization())
VGG16_model.add(Dropout(0.5))  # Add dropout with a rate of 0.5
VGG16_model.add(Dense(4, activation='softmax'))

58889256/58889256 [==============================] - 0s 0us/step


In [11]:
VGG16_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout (Dropout)           (None, 512)               0

In [14]:
learning_rate = 0.001  # Adjust the learning rate as needed
optimizer = Adam(learning_rate=learning_rate)

In [15]:
epochs=100

In [16]:
VGG16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # You can use 'accuracy' or other metrics based on your preference
    patience=10,           # Number of epochs with no improvement after which training will be stopped
    verbose=1,
    mode='min',           # Training will stop when the quantity monitored has stopped decreasing
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [ ]:
history = VGG16_model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs,
        callbacks= [early_stopping]
     )

Epoch 1/100
195/195 [==============================] - 147s 683ms/step - loss: 0.3636 - accuracy: 0.8757 - val_loss: 0.5911 - val_accuracy: 0.7869
Epoch 2/100
195/195 [==============================] - 114s 585ms/step - loss: 0.1923 - accuracy: 0.9344 - val_loss: 0.1399 - val_accuracy: 0.9594
Epoch 3/100
195/195 [==============================] - 113s 579ms/step - loss: 0.1489 - accuracy: 0.9469 - val_loss: 0.0686 - val_accuracy: 0.9742
Epoch 4/100
195/195 [==============================] - 112s 575ms/step - loss: 0.1408 - accuracy: 0.9495 - val_loss: 0.0805 - val_accuracy: 0.9742
Epoch 5/100
195/195 [==============================] - 112s 575ms/step - loss: 0.1212 - accuracy: 0.9566 - val_loss: 0.0700 - val_accuracy: 0.9762
Epoch 6/100
195/195 [==============================] - 113s 580ms/step - loss: 0.1096 - accuracy: 0.9595 - val_loss: 0.0600 - val_accuracy: 0.9813
Epoch 7/100
195/195 [==============================] - 113s 578ms/step - loss: 0.1046 - accuracy: 0.9641 - val_loss: 0

In [ ]:
steps = len(test_generator)
test_loss, test_accuracy = VGG16_model.evaluate(test_generator, steps=steps)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

predictions = VGG16_model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming `test_generator` is your test data generator
num_batches = len(test_generator)
batch_size = test_generator.batch_size

true_labels = []
predicted_labels = []
predicted_probs = []

# Generate predictions in batches
for i in range(num_batches):
    data_batch, labels_batch = next(test_generator)

    true_labels.extend(np.argmax(labels_batch, axis=1))  # Extracting class indices without calling `numpy()`

    predictions_batch = VGG16_model.predict(data_batch)
    predicted_labels.extend(np.argmax(predictions_batch, axis=1))
    predicted_probs.extend(predictions_batch)

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)
predicted_probs = np.concatenate(predicted_probs)

class_names = list(test_generator.class_indices.keys())

# Classification Report
report = classification_report(true_labels, predicted_labels, target_names=class_names)
print("Classification Report:")
print(report)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(true_labels, predicted_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Access the training history from the history object
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
# You can also access other metrics like accuracy, etc. if you have them.

# Create a plot to visualize the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')
plt.plot(range(1, len(validation_loss) + 1), validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Access the training history from the history object
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
# You can also access other metrics like accuracy, etc. if you have them.

# Create a plot to visualize the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(training_accuracy) + 1), training_accuracy, label='Training accuracy')
plt.plot(range(1, len(validation_accuracy) + 1), validation_accuracy, label='Validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.show()